<a href="https://www.kaggle.com/code/samyukthakaditham/word2vac-with-gensim-nlp-library?scriptVersionId=169198804" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import gensim
import pandas as pd

Reading and Exploring the Dataset
The dataset we are using here is a subset of Amazon reviews from the Cell Phones & Accessories category. The data is stored as a JSON file and can be read using pandas.

Link to the Dataset: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [2]:
df = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [3]:
df.shape

(194439, 9)

### Simple Preprocessing & Tokenization
The first thing to do for any data science task is to clean the data. For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. This is something we will do over here too.

Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [4]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [5]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [6]:
review_text= df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [7]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [8]:
review_text[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

## Training the Word2Vec Model
Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

#### Initialize the model

In [10]:
model= gensim.models.Word2Vec(
    window=10, # no.of Words before and after the present word
    min_count=2,  # sentence should have atleast 2 word. dont consider otherwise.
    workers= 4,   #Number of CPU Cores
)

#### Build Vocabulary

In [11]:
model.build_vocab(review_text, progress_per= 1000)

In [12]:
model.epochs # default epochs are 5

5

In [13]:
model.vector_size # default vector size is 100

100

In [14]:
model.corpus_count

194439

#### Train the Word2Vac Model

In [15]:
model.train(review_text, total_examples= model.corpus_count, epochs= model.epochs)

(61509764, 83868975)

#### Save he model
Save the model so that it can be reused in other applications

In [16]:
model.save('./word2vec-amazon-cell-accessories-reviews-short.model')

### Finding Similar Words and Similarity between words
https://radimrehurek.com/gensim/models/word2vec.html

In [17]:
model.wv.most_similar('bad')

[('terrible', 0.6782456636428833),
 ('shabby', 0.6381737589836121),
 ('horrible', 0.5840318202972412),
 ('legit', 0.5798123478889465),
 ('good', 0.5771300792694092),
 ('funny', 0.5481271743774414),
 ('awful', 0.5401236414909363),
 ('poor', 0.522925853729248),
 ('disappointing', 0.5219603776931763),
 ('keen', 0.5166008472442627)]

In [18]:
model.wv.most_similar('good')

[('decent', 0.8248418569564819),
 ('great', 0.7954105734825134),
 ('nice', 0.7048239707946777),
 ('fantastic', 0.7007414698600769),
 ('superb', 0.6454076170921326),
 ('excellent', 0.6246329545974731),
 ('outstanding', 0.609365701675415),
 ('awesome', 0.6031378507614136),
 ('phenomenal', 0.6010478138923645),
 ('reasonable', 0.5890554189682007)]

In [19]:
model.wv.similarity(w1= 'cheap', w2= 'inexpensive')

0.5234263

In [20]:
model.wv.similarity(w1= 'good', w2= 'bad')

0.5771301

In [21]:
model.wv.similarity(w1= 'good', w2= 'great')

0.79541045